In [1]:
pip install transformers pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch

C:\Users\Shauryaman\anaconda3\Lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [3]:
csv_file_path = 'myntra_products_catalog.csv'
df = pd.read_csv(csv_file_path)
print(df.head())

   ProductID                                        ProductName ProductBrand  \
0   10017413  DKNY Unisex Black & Grey Printed Medium Trolle...         DKNY   
1   10016283  EthnoVogue Women Beige & Grey Made to Measure ...   EthnoVogue   
2   10009781  SPYKAR Women Pink Alexa Super Skinny Fit High-...       SPYKAR   
3   10015921  Raymond Men Blue Self-Design Single-Breasted B...      Raymond   
4   10017833  Parx Men Brown & Off-White Slim Fit Printed Ca...         Parx   

   Gender  Price (INR)  NumImages  \
0  Unisex        11745          7   
1   Women         5810          7   
2   Women          899          7   
3     Men         5599          5   
4     Men          759          5   

                                         Description PrimaryColor  
0  Black and grey printed medium trolley bag, sec...        Black  
1  Beige & Grey made to measure kurta with churid...        Beige  
2  Pink coloured wash 5-pocket high-rise cropped ...         Pink  
3  Blue self-design band

In [4]:
print("Columns in the CSV file:", df.columns)

Columns in the CSV file: Index(['ProductID', 'ProductName', 'ProductBrand', 'Gender', 'Price (INR)',
       'NumImages', 'Description', 'PrimaryColor'],
      dtype='object')


In [5]:
text_column = ('ProductName')
if text_column not in df.columns:
    raise ValueError(f"The specified column '{text_column}' is not present in the CSV file.")

In [6]:
missing_values_count = df[text_column].isnull().sum()
if missing_values_count > 0:
    print(f"Warning: There are {missing_values_count} missing values in the '{text_column}' column. Handling missing values by filling with an empty string.")
    df[text_column].fillna('', inplace=True)

In [7]:
print(df[text_column].head())

0    DKNY Unisex Black & Grey Printed Medium Trolle...
1    EthnoVogue Women Beige & Grey Made to Measure ...
2    SPYKAR Women Pink Alexa Super Skinny Fit High-...
3    Raymond Men Blue Self-Design Single-Breasted B...
4    Parx Men Brown & Off-White Slim Fit Printed Ca...
Name: ProductName, dtype: object


In [8]:
# Cleaning the Text Data
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [9]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Shauryaman\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Shauryaman\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
csv_file_path = 'myntra_products_catalog.csv'
df = pd.read_csv(csv_file_path)
text_column = 'ProductName' # Cleaning Product name Coloumn
text_column1 = 'Description' # Cleaning the Description Coloumn

In [11]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [12]:
# Function to clean text data (Reffered From Chat gpt)
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove extra whitespace
    text = ' '.join(text.split())
    # Remove stop words
    words = text.split()
    words = [word for word in words if word not in stop_words]
    # Lemmatize words
    words = [lemmatizer.lemmatize(word) for word in words]
    # Reconstruct the text
    text = ' '.join(words)
    return text

In [13]:
df[text_column] = df[text_column].apply(preprocess_text)
df[text_column1] = df[text_column1].apply(preprocess_text)
print(df)

       ProductID                                        ProductName  \
0       10017413  dkny unisex black grey printed medium trolley bag   
1       10016283  ethnovogue woman beige grey made measure custo...   
2       10009781  spykar woman pink alexa super skinny fit highr...   
3       10015921  raymond men blue selfdesign singlebreasted ban...   
4       10017833  parx men brown offwhite slim fit printed casua...   
...          ...                                                ...   
12486   10262843  pepe jean men black hammock slim fit lowrise c...   
12487   10261721                   mochi woman goldtoned solid heel   
12488   10261607  league girl navy blue white printed regular fi...   
12489   10266621  bvlgari men aqva pour homme marine eau de toil...   
12490   10265199  pepe jean men black grey striped polo collar t...   

      ProductBrand  Gender  Price (INR)  NumImages  \
0             DKNY  Unisex        11745          7   
1       EthnoVogue   Women         5810

In [14]:
df
text_column = 'ProductName'

In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [16]:
def encode_texts(texts, tokenizer, max_length=512):
    # Tokenize and encode texts
    encoded_dict = tokenizer.batch_encode_plus(
        texts,
        add_special_tokens=True,  # Add [CLS] and [SEP]
        max_length=max_length,  # Pad or truncate to max_length
        padding='max_length',  # Pad to max_length
        truncation=True,  # Truncate to max_length
        return_attention_mask=True,  # Return attention masks
        return_tensors='pt',  # Return PyTorch tensors
    )
    return encoded_dict

In [17]:
texts = df[text_column].tolist()
encoded_dict = encode_texts(texts, tokenizer)
input_ids = encoded_dict['input_ids']
attention_masks = encoded_dict['attention_mask']
print("Input IDs shape:", input_ids.shape)
print("Attention Masks shape:", attention_masks.shape)

Input IDs shape: torch.Size([12491, 512])
Attention Masks shape: torch.Size([12491, 512])


In [18]:
df
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [19]:
def encode_text(text):
    # Tokenize input text
    tokens = tokenizer.encode(text, add_special_tokens=True)
    # Convert tokens to tensor
    tokens_tensor = torch.tensor(tokens)
    # Reshape tensor for compatibility with BERT model
    tokens_tensor = tokens_tensor.unsqueeze(0)
    return tokens_tensor

In [20]:
encoded_texts = []
for text in df['ProductName']:
    encoded_text = encode_text(text)
    encoded_texts.append(encoded_text)

In [21]:
encoded_texts_tensor = torch.cat(encoded_texts, dim=1)

In [22]:
model = BertModel.from_pretrained('bert-base-uncased')

In [28]:
torch.save(model, 'encoded_data.csv')